In [1]:
from datetime import datetime
from concurrent import futures

import pandas as pd
import numpy as np
from pandas import DataFrame
import pandas_datareader.data as web
from pandas_datareader import data

import time,datetime
import glob
import fix_yahoo_finance as yf
yf.pdr_override()


from informs import portfolio
import class_informs as inform
import generator
import discriminator

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm

import cmath

import re
import matplotlib.pyplot as plt

In [2]:
SP500=pd.read_csv('data/s&p500.csv')

SP100=pd.read_csv('data/s&p100.csv')

SP400=pd.read_csv('data/s&p400.csv', encoding= "ISO-8859-1")
SP400=SP400.rename(columns={'Ticker Symbol' : 'Symbol'})

FTSE500=pd.read_csv('data/FTSE500.csv')
FTSE500=FTSE500.rename(columns={'Ticker': 'Symbol'})
FTSE500=FTSE500.rename(columns={'FTSE Sector[9]': 'Sector'})

DAX=pd.read_csv('data/DAX.csv')
DAX=DAX.rename(columns={'Ticker symbol.1' : 'Symbol'})

NASDAQ100=pd.read_csv('data/NASDAQ100.csv')

NYSE=pd.read_csv('data/NYSE.csv')

Russell1000 = pd.read_csv('data/Russell1000.csv')
Russell1000 = Russell1000.rename(columns={'Ticker' : 'Symbol'})


In [50]:
Russell2000 = pd.read_csv('data/Russell2000.csv')

Russell2000 = Russell2000.rename(columns={'Ticker' : 'Symbol'})

# Russell2000

In [105]:
EWY = pd.read_csv('data/EWY.csv')

sym=[]
for i in list(EWY['Ticker']):
    if len(str(i)) < 6:
        sym.append('0'*(6-len(str(i)))+str(i)+'.KS')
    else:
        sym.append(str(i)+'.KS')
        
EWY['Symbol'] = sym


In [121]:
Russell1000_2 = pd.read_csv('data/Russell1000_2.csv')
Russell1000_2 = Russell1000_2.rename(columns={'Ticker' : 'Symbol'})


In [117]:
a

954

In [119]:
Russell1000_2

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange
0,AAPL,APPLE INC,Equity,3.55,178.12,"3,980,446.00","708,997,041.52","708,997,041.52",Information Technology,2046251,US0378331005,NASDAQ
1,MSFT,MICROSOFT CORP,Equity,2.73,93.77,"5,810,553.00","544,855,554.81","544,855,554.81",Information Technology,2588173,US5949181045,NASDAQ
2,AMZN,AMAZON COM INC,Equity,2.33,"1,512.45","307,803.00","465,536,647.35","465,536,647.35",Consumer Discretionary,2000019,US0231351067,NASDAQ
3,FB,FACEBOOK CLASS A INC,Equity,1.62,178.32,"1,808,114.00","322,422,888.48","322,422,888.48",Information Technology,B7TL820,US30303M1027,NASDAQ
4,JPM,JPMORGAN CHASE & CO,Equity,1.55,115.5,"2,675,664.00","309,039,192.00","309,039,192.00",Financials,2190385,US46625H1005,New York Stock Exchange Inc.
5,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Equity,1.55,207.2,"1,490,318.00","308,793,889.60","308,793,889.60",Financials,2073390,US0846707026,New York Stock Exchange Inc.
6,JNJ,JOHNSON & JOHNSON,Equity,1.36,129.88,"2,089,192.00","271,344,256.96","271,344,256.96",Health Care,2475833,US4781601046,New York Stock Exchange Inc.
7,GOOG,ALPHABET INC CLASS C,Equity,1.30,"1,104.73","234,232.00","258,763,117.36","258,763,117.36",Information Technology,BYY88Y7,US02079K1079,NASDAQ
8,GOOGL,ALPHABET INC CLASS A,Equity,1.28,"1,103.92","230,581.00","254,542,977.52","254,542,977.52",Information Technology,BYVY8G0,US02079K3059,NASDAQ
9,XOM,EXXON MOBIL CORP,Equity,1.25,75.74,"3,284,584.00","248,774,392.16","248,774,392.16",Energy,2326618,US30231G1022,New York Stock Exchange Inc.


In [120]:

def get_stockdata(stocks,start_date,end_date):
    
    fail_names =[] 

    df=pd.DataFrame()
    
    for stock in stocks:
        try:
#             stock_df = data.get_data_yahoo(stock, start_date, end_date)
            stock_df=data.YahooDailyReader(stock,start_date,end_date).read()
            stock_df['Name'] = stock
            df=df.append(stock_df)
        except:
            
            print('bad: %s' % (stock))
            try:
                fail_names=fail_names.append(stock)
            except:
                pass

            
    return df,fail_names


def bench(df,NEW_stocks):
        
    THIS=df[df.index==df.index[-2]]

    THIS['tot_market']=THIS['Close']*THIS['Volume']
    THIS['bench']=THIS['tot_market']/THIS['tot_market'].sum()

    dic_bench={}
    for i in list(THIS['Name']):
        dic_bench[i]=float(THIS[THIS['Name']==i]['bench'])

    return dic_bench,THIS



def sector_mcapq(THIS,INFO,NEW_stocks):

    THIS=pd.merge(THIS, INFO, how='left')
#     THIS=THIS.rename(columns={'GICS_Sector' : 'Sector'})
    THIS['rank'] = THIS['bench'].rank(ascending=0,method='min')
    THIS['mcapq'] = np.where(THIS['rank']<=len(NEW_stocks)*0.2, '1', np.where(THIS['rank']<=len(NEW_stocks)*0.4, '2', np.where(THIS['rank']<=len(NEW_stocks)*0.6, '3', np.where(THIS['rank']<=len(NEW_stocks)*0.8, '4', '5'))))

    THIS = THIS.replace(np.nan, 'Nan', regex=True)
    dic_sector = {THIS["Sector"][i] : [] for i in THIS.index }

    dic_MCAP = {THIS["mcapq"][i] : [] for i in THIS.index}

    for i in THIS.index:
        dic_sector[THIS["Sector"][i]].append(THIS["Name"][i])
        dic_MCAP[THIS["mcapq"][i]].append(THIS["Name"][i])
    THIS['index']=list(THIS.index)
    
    return dic_sector,dic_MCAP,THIS
        
        
def omega_covariances(prices):
    prices =np.matrix(prices)
    rows, cols = prices.shape
    returns = np.empty([rows, cols - 1])
    for r in range(rows):
        for c in range(cols - 1):
            p0, p1 = prices[r, c], prices[r, c + 1]
            returns[r, c] = (p1 / p0) - 1
    covars = np.cov(returns)
    covars = covars * len(prices[0]) 
    return covars


def alpha_return(stocks,start_date,end_date):
    
    close = pd.DataFrame()
    fail_names=[]

    for stock in stocks:
        try:
            close[stock] =  data.get_data_yahoo(stock, start_date, end_date)['Adj Close']
        except:
            fail_names.append(stock)

    daily_simple_returns = close.pct_change()
    annual_returns = daily_simple_returns.mean() * len(close)
    
    return close,annual_returns,fail_names



In [ ]:
############## (INDEX , stocks, INFO) are required to be changed passively ##################


##### Data Setting ######
INDEX = 'Russell1000_2'
start_date = '2017-02-16'
end_date = '2018-02-15'
# df=pd.read_csv('data/NYSE_df.csv')
# stocks= list(set(NYSE['Symbol'])-set(df[df['Volume']==0]['Name'].unique()) -set(df[df['Close']>1000]['Name'].unique()))
stocks = list(set(Russell1000_2['Symbol']))


##### Data Loading #####
df,fail_name=get_stockdata(stocks,start_date,end_date)
df.to_csv('./data/%s_df.csv'%(INDEX))


##### Omega #####
prices=list(df.groupby('Name')['Close'].apply(list))
print(prices)
date_len=len(df.index.unique().tolist())
for i in range(len(prices)):
    if len(prices[i]) != date_len:
        prices[i].extend([prices[i][-1]]*(date_len-len(prices[i])))
        
risk_mat = omega_covariances(prices)
NEW_stocks=list(df.groupby('Name')['Close'].describe().index)
        #risk_sedol->NEW_stocks

##### Alpha ####
close,dic_alpha,fail_names2 = alpha_return(NEW_stocks,start_date,end_date)
        #dic_sedol_as->dic_alpha    
    

##### Bench Weight #####
dic_bench,THIS = bench(df,NEW_stocks)


##### sector, MCAPQ #####

# FTSE500=FTSE500.rename(columns={'Symbol': 'Name'})
# INFO=FTSE500.rename(columns={'FTSE Sector[9]': 'Sector'})


INFO=pd.read_csv('data/s&p500 list.csv')
INFO=INFO.rename(columns={'Ticker symbol' : 'Name'})
INFO=INFO.rename(columns={'GICS_Sector' : 'Sector'})

dic_sector,dic_MCAP,THIS = sector_mcapq(THIS,INFO,NEW_stocks)



######### Save #########

with open('all_data_%s.txt'%(INDEX), 'wb') as f:
    pickle.dump(risk_mat, f)
    pickle.dump(NEW_stocks, f)
    pickle.dump(close, f)
    pickle.dump(dic_alpha, f)
    pickle.dump(dic_bench, f)
    pickle.dump(dic_sector, f)
    pickle.dump(dic_MCAP, f)
    pickle.dump(THIS, f)



bad: GLPI
bad: WPC
bad: BIO
bad: TIF
bad: MIK
bad: CAG
bad: BRX
bad: QRVO
bad: BDX
bad: MS
bad: WM
bad: LW
bad: JNPR
bad: BERY
bad: APH
bad: GWW
bad: AMG
bad: MNST
bad: AMCX
bad: BRKB
bad: ALLE
bad: UDR
bad: M
bad: FANG
bad: MSCI
bad: APLE
bad: LGFA
bad: CAH
bad: UBFUT
bad: DPZ
bad: BIIB
bad: FLIR
bad: MAR
bad: PRU
bad: ACM
bad: DCT
bad: LKQ


In [93]:
risk_mat.min()

-0.00013746935024824426

In [207]:
risk_mat.min()

-0.071903402118972046